# Energy guided EOT (generalization)
**author**: *Sergey Karpukhin*, sergey.karpukhin@skoltech.ru tg: @hr3nk

## 1. Theory

### 1.1 Energy Based Models

#### Definition

Our desire is to sample from true data distribution $p(x)$, but since we don't have access to all possible data $X$, we are limited to approximating true data distribution with model $p_\theta(x)$. EBM is a way to learn such approximation.

Let $E_\theta: \mathbb{R}^D \rightarrow \mathbb{R^{+}}$ be a neural network that maps data to scalar values greater or equal then zero. This neural network will be called *energy function*. Then we can define model of our probability distribution on our data as following:

$$
p_\theta(x) = \frac{1}{z_\theta} \exp(-E_\theta(x)),
$$

where $z_\theta$ is a normalization constant equial to $\int_X \exp(-E_\theta(x)) \mathrm{d}x$ and generally is not computable.

#### Training of EBM

To train EBM model we need to formulate optimization goal, which can be done either through $KL$-divergence between approximation and ground truth distribution or maximum-likelihood (they are equivalent for optimization).

$$
KL(p_{\theta} \| p) \rightarrow \min_{\theta}
$$

Derivation of loss function:

$$
KL(p_{\theta} \| p) = \int_{\mathcal{X}} \log p(x) \mathrm{d}p - \int_{\mathcal{X}} \log p_\theta(x) \mathrm{d}p
$$

First term can be dropped since it doesn't depend on $\theta$ and it want affect our optimization.

$$
\mathcal{L}(\theta; X) = \int_{\mathcal{X}} \log p_\theta(x) \mathrm{d}p   = -\int_{\mathcal{X}}\log(z_\theta)\mathrm{d}p - \int_{\mathcal{X}}E_\theta(x) \mathrm{d}p = \\
= -\log(z_\theta) - \int_{\mathcal{X}}E_\theta(x) \mathrm{d}p \rightarrow \max_\theta
$$

As we are ultimately training neural network, in order to do gradient update of model parameters, we need to compute loss gradient:

$$
\nabla_{\theta} \mathcal{L}(\theta; X) = -\nabla_\theta\log(z_\theta)- \int_{\mathcal{X}}\nabla_\theta E_\theta(x) \mathrm{d}p
$$

Second term is straight-forward to compute via sampling from data at hand. Let's examine $\nabla_{\theta} \log(z_\theta)$ separately.

$$
\nabla_\theta \log(z_\theta)  = \frac{1}{z_\theta} \nabla_\theta z_\theta = \\
= \frac{1}{z_\theta} \int_X \nabla_\theta \exp(-E_\theta(x))\mathrm{d}x = \\
= \int_X -\nabla_\theta E_\theta(x) \frac{1}{z_\theta} \exp(-E_\theta(x))\mathrm{d}x\\
= \int_X -\nabla_\theta E_\theta(x) p_\theta(x)\mathrm{d}x \\
= - \int_\mathcal{X}\nabla_\theta E_\theta(x) \mathrm{d}p_\theta
$$

Finally, combining both terms, we obtain:

$$
\nabla_\theta \mathcal{L}(\theta; X) = \int_\mathcal{X}\nabla_\theta E_\theta(x) \mathrm{d}p_\theta - \int_{\mathcal{X}}\nabla_\theta E_\theta(x) \mathrm{d}p
$$

First integral can be estimated via sampling from learned $p_\theta$ (see Inference for details on sampling). Therefore learning algorithm of EBM looks like this:

- $E_\theta$ - model
1. $\{x_{i}\}^N_{i=1} \sim p(x)$ - samples from dataset
2. $\{\hat x_{i}\}^N_{i=1} \sim p_\theta(x)$ - samples from approximated distribution via Langevin dynamics
3. $\nabla_\theta \mathcal{L} = \frac{1}{N}\sum_i^M \nabla_\theta E_\theta(\hat x_{i}) - \frac{1}{N} \sum_i^N \nabla_\theta E_\theta(x)$ - simply computed by auto-grad
4. $\theta_{k+1} = \theta_{k} + a \nabla_\theta \mathcal{L}$ - update parameters via gradient ascent (for descent need to negate the loss function)

#### EBM Sampling
By sampling from EBM we mean that parameters of the model we want to obtain samples from our approximated distribution defined by energy function. We know $p_\theta(x)$ up to a constant, so for sampling we can apply **Langevin dynamics**, which relies on gradient of our energy function.

Algorithm for sampling:
- $E_\theta$ - trained_model, $K$ - number of iterations, $n$ - discetization step
1. $x_0 \sim \mathcal{N}(0, I_D)$
2. for $i$ in [$1 \dots K$]:
3. $~~~~x_{i} = x_{i-1} - \frac{n^2}{2} \nabla_x E_\theta(x) + n \mathcal{N}(0, I_D)$
4. return $x_{K}$

### 1.2 EBM For Entropic Optimal Transport


**Connecting EBM and EOT**

Our objective - optimal plan between $p(x)$ and $q(y)$, can be formulated as a following approximation (which nicely coincides with EBM defintion).

$$
\pi_\theta(x, y) = \frac{1}{z_\theta} \exp(u_\theta(x) + v_\theta(y) - c(x, y))
$$

Then, repeating same derivations given in $EBM$ section, we can formulate optimization goal as $KL(\pi_\theta | \pi^*)$

$$
KL(\pi_\theta | \pi^*) = \int \log (\pi^*(x, y))d\pi^* - \int \log (\pi_\theta(x, y)) \mathrm{d}\pi^*(x, y)
$$

First term is const w.r.t $\theta$, so we drop it in later derivations.

$$
\mathcal{L}(\theta; X, Y) = \int \log(\pi_\theta(x, y))\mathrm{d}\pi^*(x, y) = - \log z_\theta + \int (u_\theta(x) + v_\theta(y) - c(x, y))\mathrm{\pi^*(x, y)} = \\ = - \log z_\theta + \int u_\theta(x)\mathrm{\pi^*(x, y)} +  \int v_\theta(y)\mathrm{\pi^*(x, y)} -  \int c(x, y)\mathrm{\pi^*(x, y)} = \\ = - \log z_\theta + \int u_\theta(x)\mathrm{d}\mathbb{P}(x) +  \int v_\theta(y)\mathrm{d}\mathbb{Q}(y)
$$

Last transition follows from known marginals $\pi^*(x, y)$: $\int_X \pi^*(x, y)dx = \mathbb{Q}(y)$ and $\int_Y \pi^*(x, y)dy = \mathbb{P}(x)$. So from disintegration theorem $\mathrm{d}\pi^*(x, y) = \mathrm{d}\pi^*(x | y) \mathrm d\mathbb{Q}(y) = \mathrm{d}\pi^*(y | x) \mathrm d\mathbb{P}(x)$

Yet again, let's compute the $\nabla_\theta(\cdot)$

$$
\nabla_\theta \mathcal{L} = - \nabla_\theta \log z_\theta + \int \nabla_\theta u_\theta(x)\mathrm{d}\mathbb{P}(x) +  \int \nabla_\theta v_\theta(y)\mathrm{d}\mathbb{Q}(y) = \\ = \int \nabla_\theta u_\theta(x)\mathrm{d}\mathbb{P}(x) +  \int \nabla_\theta v_\theta(y)\mathrm{d}\mathbb{Q}(y) - \int (\nabla_\theta u_\theta(x) + \nabla_\theta v_\theta(y))d\pi_\theta(x,y)
$$

which yields a very close expression the one from EBM section. However here we are required to sample from joint distribution defined by $\pi_\theta$.

Note: this expression is derived for maximization of likelihood, so in actual training regime sign is inversed, since torch only knows gradient descent.


**Details on Langevin dynamics for new parametrization**

Recall, in original langevin dynamics formulation we use $\nabla_x \log p(x)$ of the target distribution we are sampling from. Since our distribution is a joint plan, we need to calculate $\nabla_{(x, y)} \pi_\theta()$. Since parametrization is splitted between two potential functions - $u(x)$ and $v(x)$, we can easily calculate partial derivative for each dimension:

$$
\frac{\partial \pi_\theta(x, y)}{\partial x} = \frac{\partial}{\partial}u(x) - \frac{\partial}{\partial x}c(x, y) \\
\frac{\partial \pi_\theta(x, y)}{\partial y} = \frac{\partial}{\partial y}v(y) - \frac{\partial}{\partial y}c(x, y)
$$

And that is precisely formulas used in updates for algorithm described in algorithm for sampling from EBM.